# Photonic VQE Tutorial: Simulating Molecular Ground States with Qlass

This tutorial demonstrates how to use the Qlass library to perform Variational Quantum Eigensolver (VQE) simulations for molecular systems using photonic circuits. We start with the H2 molecule and discuss how to extend to more complex molecules like LiH. The notebook is updated for the latest `qlass.vqe` module and is well-commented for new users.

## Introduction

The Variational Quantum Eigensolver (VQE) is a hybrid quantum-classical algorithm used to estimate the ground state energy of quantum systems, such as molecules. In this tutorial, we:
- Import required libraries
- Load molecular Hamiltonian data from file
- Define the VQE ansatz and executor
- Run VQE for H2 and discuss how to extend to LiH
- Compare results with theoretical values

In [ ]:
# Import required libraries
import perceval as pcvl
import perceval.components as comp
from perceval.algorithm import Sampler

from qlass.utils import loss_function, rotate_qubits
from qlass.vqe import le_ansatz

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

from scipy.optimize import minimize
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

# Import H2 Hamiltonian data from file
from examples.h2_hamiltonian_data import Hamiltonian_elem, Hamiltonian_coef

## Hamiltonian Construction

We use precomputed Hamiltonian elements and coefficients for the H2 molecule, loaded from file. The Hamiltonian is constructed for each bond length (radius) as a linear combination of Pauli operators. Theoretical ground state energies are also computed for comparison.

In [ ]:
# Build the Hamiltonian for each radius
H1 = []
(n, m) = Hamiltonian_coef.shape
for i in range(n):  # i = Radius
    h_0 = 1.0 * np.matrix(Hamiltonian_elem[0])
    for j in range(1, m):
        h_0 += Hamiltonian_coef[i, j] * np.matrix(Hamiltonian_elem[j])
    H1.append([Hamiltonian_coef[i, 0], h_0])
H = H1

# Compute theoretical ground state energies
E1_th = []
for h in H:
    l0 = np.linalg.eigvals(h[1])
    l0.sort()
    E1_th.append(min(l0))

def hamiltonian_dictionary(h):
    """
    Convert a 2-qubit Hamiltonian from array form to a dictionary.
    Args:
        h (np.ndarray): 2-qubit Hamiltonian in array form
    Returns:
        Dict[str, float]: Dictionary with Pauli string keys and coefficient values
    """
    pauli_strings = ["II", "IX", "IZ", "XI", "XX", "XZ", "ZI", "ZX", "ZZ"]
    return dict(zip(pauli_strings, h))

## VQE Simulation with Qiskit Ansatz

We use a Qiskit-style ansatz and Perceval's Sampler to estimate the ground state energy for each bond length. The executor function defines how the ansatz is constructed and sampled.

In [ ]:
# Executor for a two-qubit Qiskit ansatz

def executor(lp, pauli_string):
    processor = le_ansatz(lp, pauli_string)
    sampler = Sampler(processor)
    samples = sampler.samples(10_000)
    return samples

In [ ]:
# Run VQE for H2 molecule
radius1 = []
E1_qiskit = []
num_qubits = 2
for R in tqdm(range(5, 50, 2)):
    h = np.array(Hamiltonian_coef[R])[0][1:]
    H = hamiltonian_dictionary(h)
    radius1.append(np.array(Hamiltonian_coef[R])[0][0])
    initial_params = np.random.rand(2*num_qubits)
    result = minimize(loss_function, initial_params, args=(H, executor), method='COBYLA', options={'maxiter': 30})
    E1_qiskit.append(result.get('fun'))

In [ ]:
# Plot results for H2
plt.plot(100*np.array(radius1), np.array(E1_th)[range(5, 50, 2)],'orange')
plt.plot(100*np.array(radius1), E1_qiskit,'x')
plt.ylabel('Energy (MJ/mol)')
plt.xlabel('Atomic separation R (pm)')
plt.legend(['Theoretical ground energies', 'Ground energies computed with Perceval'])
plt.show()

min_value = min(E1_qiskit)
min_index = E1_qiskit.index(min_value)
print(f'The minimum energy is E_g({radius1[min_index]})={E1_qiskit[min_index]} MJ/mol and is attained for R_min ={radius1[min_index]} pm')

## Generalizing to LiH and Other Molecules

To expand this notebook to LiH, you would need to:
- Obtain or compute the Hamiltonian elements and coefficients for LiH (e.g., using quantum chemistry packages)
- Save them in a similar format as the H2 data file
- Update the import and Hamiltonian construction steps accordingly

The VQE workflow remains the same, but the number of qubits and ansatz complexity may increase. For larger molecules, consider more advanced ansatz circuits and optimization strategies.

## Comments on Results

- The VQE results for H2 closely follow the theoretical ground state energies, demonstrating the effectiveness of the approach.
- For LiH and larger molecules, expect increased computational complexity and the need for more sophisticated ansatz and optimization.
- The modular structure of this notebook allows easy extension to new molecules and ansatz types.

---
**End of Tutorial**